<a href="https://colab.research.google.com/github/sayantan-2/comfyui_colab_Flux/blob/main/comfyui_colab_Flux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Git clone the [ComfyUI](https://github.com/comfyanonymous/ComfyUI) repo and install the requirements.

In [ ]:
!git clone -q https://github.com/comfyanonymous/ComfyUI
%pip install -q xformers!=0.0.18 -r /content/ComfyUI/requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

In [2]:
# Some Helpful Custom Nodes
!git clone -q https://github.com/ltdrdata/ComfyUI-Manager /content/ComfyUI/custom_nodes/comfyui-manager

!git clone -q https://github.com/ClownsharkBatwing/RES4LYF.git /content/ComfyUI/custom_nodes/RES4LYF
!git clone -q https://github.com/rgthree/rgthree-comfy.git /content/ComfyUI/custom_nodes/rgthree-comfy
!git clone -q https://github.com/yolain/ComfyUI-Easy-Use.git /content/ComfyUI/custom_nodes/ComfyUI-Easy-Use

Download the vae and dual text encoder needed for FLUX - (Compulsory)

In [ ]:
!wget -q --show-progress https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors -P /content/ComfyUI/models/text_encoders/
!wget -q --show-progress https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp8_e4m3fn.safetensors -P /content/ComfyUI/models/text_encoders/
!wget -q --show-progress https://huggingface.co/cocktailpeanut/xulf-dev/resolve/main/ae.safetensors -P /content/ComfyUI/models/vae/

# 8-step distilled lora for fast inference on dev models
# !wget -q --show-progress https://huggingface.co/alimama-creative/FLUX.1-Turbo-Alpha/resolve/main/diffusion_pytorch_model.safetensors -P /content/ComfyUI/models/loras


The **FP8 version** runs smoothly on the free tier (without LoRA), but if you want to use it with a LoRA, it requires more resources than the free Colab tier provides.  
In that case, you can opt for the [gguf](https://huggingface.co/collections/QuantStack/flux-ggufs-68264cfc663d50c418940b30) quantized versions or the [Nunchaku](https://huggingface.co/nunchaku-tech/models) versions instead.

>**Note** : You need only one , either a gguf quantized model (under 10GB) or the Nunchanku model

---


### Option 1 : fp8 quantized - from [Comfy-Org](https://huggingface.co/Comfy-Org/flux1-dev/tree/main)

In [ ]:
# fp8 - needs more than free tier
# !wget -q --show-progress https://huggingface.co/Comfy-Org/flux1-dev/resolve/main/flux1-dev-fp8.safetensors -P /content/ComfyUI/models/checkpoints

### Option 2 : [**gguf**](https://huggingface.co/city96/FLUX.1-dev-gguf) models.  

Git clone the [ComfyUI-GGUF](https://github.com/city96/ComfyUI-GGUF) repo and follow the instructions.

In [ ]:
# %pip install --upgrade gguf
# !rm -rf ./custom_nodes/ComfyUI-GGUF
# !git clone https://github.com/city96/ComfyUI-GGUF /content/ComfyUI/custom_nodes/ComfyUI-GGUF

In [ ]:
# Download the one you need

# flux1-dev
# !wget -q --show-progress https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q6_K.gguf -P /content/ComfyUI/models/unet

# flux1-kontext-dev
# !wget -q --show-progress https://huggingface.co/QuantStack/FLUX.1-Kontext-dev-GGUF/resolve/main/flux1-kontext-dev-Q6_K.gguf -P /content/ComfyUI/models/unet

# flux1-krea-dev
# !wget -q --show-progress https://huggingface.co/QuantStack/FLUX.1-Krea-dev-GGUF/resolve/main/flux1-krea-dev-Q6_K.gguf -P /content/ComfyUI/models/unet

# flux1-schnell
# !wget -q --show-progress https://huggingface.co/city96/FLUX.1-schnell-gguf/resolve/main/flux1-schnell-Q6_K.gguf -P /content/ComfyUI/models/unet

### Option 3 : [**Nunchaku**](https://nunchaku.tech/docs/ComfyUI-nunchaku/workflows/t2i.html#nunchaku-flux-1-dev-json) models - recommended

In [ ]:
import torch
print(torch.__version__)


In [ ]:
!git clone -q https://github.com/mit-han-lab/ComfyUI-nunchaku /content/ComfyUI/custom_nodes/nunchaku_nodes
!pip install -q https://github.com/nunchaku-tech/nunchaku/releases/download/v0.3.2/nunchaku-0.3.2+torch2.8-cp311-cp311-linux_x86_64.whl

In [ ]:
# flux1-dev
!wget -q --show-progress https://huggingface.co/nunchaku-tech/nunchaku-flux.1-dev/resolve/main/svdq-int4_r32-flux.1-dev.safetensors -P /content/ComfyUI/models/diffusion_models

#flux1-krea-dev
# !wget -q --show-progress https://huggingface.co/nunchaku-tech/nunchaku-flux.1-krea-dev/resolve/main/svdq-int4_r32-flux.1-krea-dev.safetensors -P /content/ComfyUI/models/diffusion_models

#flux1-schnell
# !wget -q --show-progress https://huggingface.co/nunchaku-tech/nunchaku-flux.1-schnell/resolve/main/svdq-int4_r32-flux.1-schnell.safetensors -P /content/ComfyUI/models/diffusion_models

#flux1-kontext-dev
# !wget -q --show-progress https://huggingface.co/nunchaku-tech/nunchaku-flux.1-kontext-dev/resolve/main/svdq-int4_r32-flux.1-kontext-dev.safetensors -P /content/ComfyUI/models/diffusion_models

### Run ComfyUI with ngrok (recommended way)

In [ ]:
!pip install -q pyngrok

from pyngrok import ngrok
import subprocess, socket, time

# Set ngrok token
!ngrok config add-authtoken YOUR_TOKEN

# Start ComfyUI in background
subprocess.Popen(["python", "/content/ComfyUI/main.py", "--dont-print-server"])

# Wait until port is open
port = 8188
while True:
    try:
        sock = socket.create_connection(("127.0.0.1", port), timeout=2)
        sock.close()
        print("✅ ComfyUI server is running on port", port)
        break
    except OSError:
        print("⏳ Waiting for ComfyUI to start...")
        time.sleep(2)

# Start ngrok tunnel
public_url = ngrok.connect(8188, bind_tls=True)
print("🌐 Public URL:", public_url)


### Run ComfyUI with localtunnel

In [ ]:
# !npm install -g localtunnel

In [ ]:
# import subprocess
# import threading
# import time
# import socket
# import urllib.request
# import sys

# def wait_for_port(port):
#     while True:
#         time.sleep(0.5)
#         sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#         if sock.connect_ex(('127.0.0.1', port)) == 0:
#             sock.close()
#             break
#         sock.close()

#     print("\nComfyUI finished loading, launching localtunnel...\n")
#     print("Your public IP is:",
#           urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip())

#     p = subprocess.Popen(["lt", "--port", str(port)], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
#     for line in iter(p.stdout.readline, ''):
#         sys.stdout.write(line)
#         sys.stdout.flush()

# threading.Thread(target=wait_for_port, args=(8188,), daemon=True).start()

# subprocess.run(["python", "/content/ComfyUI/main.py"])
